In [2]:
# calculation of predicted numbers of sources in table 5
import numpy as np
import pandas as pd

from astropy.io import fits

import auxil_ML as aux


In [3]:
cat = '3FGL'
if cat == '4FGL':
    #cat_name = '4FGL'
    cat_name = '4FGL-DR2'
else:
    cat_name = cat
    
nclasses = 3

folder = '../SOM'
#folder = '../ML_3FGL_paper_extra/data/python files/catas'
cat_fn = '%s/%s_prob_catalog_%iclasses.csv' % (folder, cat_name, nclasses)
cat_df = pd.read_csv(cat_fn, index_col=0)
labels = ['AGN', 'PSR', 'MIXED']
if nclasses == 3:
    labels = ['AGN', 'PSR', 'OTHER', 'MIXED']
res_dict = {}
res_dict['uncorrected'] = {}
res_dict['corrected'] = {}

# unweighted algorithms
algs_u = ['NN', 'LR', 'RF', 'BDT']
# oversampled algorithms
algs_o = ['NN_O', 'LR_O', 'RF_O', 'BDT_O']

algs = algs_u + algs_o


In [4]:
if cat_name == '4FGL-DR2':
    fn = 'data/gll_psc_v27.fit'
elif cat_name == '4FGL':
    fn = 'data/gll_psc_v22.fit'
elif cat_name == '3FGL':
    fn = '../PS_comparison/gll_psc_v16.fit'
    
with fits.open(fn) as data:
    data[1].header
    cat0_df = aux.hdu2df(data[1], index_name='Source_Name')

print(len(cat0_df), len(cat_df))
names0 = list(cat0_df.index)
for name in cat_df.index:
    names0.remove(name)
print(len(names0))

3034 3034
0


In [5]:
#cat0_df.columns

In [6]:
if 0:
    fn = '../ML_3FGL_paper_extra/data/python files/catas/%s_missing_values_src_test.csv' % cat_name
    print(fn)
    miss_df.to_csv(fn, index_label='%s_Source_Name' % cat)

In [7]:
unass_mask = cat_df['Category_%s' % cat] == 'UNAS'
ass_mask = cat_df['Category_%s' % cat] != 'UNAS'
other_mask = cat_df['Category_%s' % cat] == 'OTHER'
n_unas = np.sum(unass_mask)
n_as = np.sum(ass_mask)
print(n_as, n_unas)

2024 1010


In [8]:
set(cat_df['Category_%s' % cat])

{'AGN', 'OTHER', 'PSR', 'UNAS'}

In [9]:
print('%s number of pulsars: %i' % (cat, np.sum(cat_df['Category_%s' % cat] == 'PSR')))
print('%s number of AGNs: %i' % (cat, np.sum(cat_df['Category_%s' % cat] == 'AGN')))

3FGL number of pulsars: 167
3FGL number of AGNs: 1744


In [21]:
print('Predictions for Table 5 (%s, %i classes)' % (cat, nclasses))
for label in labels:
    res_dict['uncorrected'][label] = np.sum((cat_df['Category_Prob'] == label) & unass_mask)
    print('%s: %i' % (label, res_dict['uncorrected'][label]))
print('Total:', n_unas)

Predictions for Table 5 (3FGL, 3 classes)
AGN: 587
PSR: 53
OTHER: 69
MIXED: 301
Total: 1010


In [11]:
# outliers
for src in ['AGN', 'PSR']:
    total = 1. * sum((cat_df['Category_%s' % cat] == src))
    nmixed = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] == 'MIXED'))
    nwrong = sum((cat_df['Category_%s' % cat] == src) & (cat_df['Category_Prob'] != src)) - nmixed
    print('%s %s (%i total)' %(cat, src, total))
    print('Mixed: %i (%.3f)' % (nmixed, nmixed/total))
    print('Wrong: %i (%.3f)' % (nwrong, nwrong/total))

3FGL AGN (1744 total)
Mixed: 57 (0.033)
Wrong: 1 (0.001)
3FGL PSR (167 total)
Mixed: 51 (0.305)
Wrong: 19 (0.114)


In [12]:
sum((cat_df['Category_%s' % cat] == 'PSR') & (cat_df['Category_Prob'] == 'AGN'))

8

In [13]:
print('4FGL-DR2 number of sources:', 5064 + 723)
print(len(cat_df))
print(len(cat0_df))

4FGL-DR2 number of sources: 5787
3034
3034


In [14]:
for label in labels:
    n_other = np.sum((cat_df['Category_Prob'] == label) & other_mask)
    n_label = 1. * np.sum((cat_df['Category_Prob'] == label) & ass_mask)
    print('%s other: %i' % (label, n_other))
    if n_label:
        print('%s other / n assoc: %.2f' % (label, n_other / n_label))
    n_corr = 1. * n_other * n_unas / n_as
    res_dict['corrected'][label] = res_dict['uncorrected'][label] - n_corr
    print('%s uncorrected: %i' % (label, res_dict['uncorrected'][label]))
    print('%s corrected: %.1f' % (label, res_dict['corrected'][label]))
print('Assoc other:', sum(other_mask))
print('Unassoc other model:', (sum(other_mask) * n_unas / n_as))

AGN other: 14
AGN other / n assoc: 0.01
AGN uncorrected: 587
AGN corrected: 580.0
PSR other: 11
PSR other / n assoc: 0.10
PSR uncorrected: 53
PSR corrected: 47.5
OTHER other: 33
OTHER other / n assoc: 0.75
OTHER uncorrected: 69
OTHER corrected: 52.5
MIXED other: 55
MIXED other / n assoc: 0.34
MIXED uncorrected: 301
MIXED corrected: 273.6
Assoc other: 113
Unassoc other model: 56.38833992094862


In [15]:
# predictions based on probabilities
noc = 0.
for pop in ['AGN', 'PSR', 'OTHER'][:nclasses]:
    pred = []
    if nclasses == 2:
        pred_corr = []
    for alg in algs:
        label = '%s_%s' % (pop, alg)
        n0 = np.sum(cat_df.loc[unass_mask, label])
        pred.append(n0)
        if nclasses == 2:
            # correct for other    
            pop_other = np.sum(cat_df.loc[other_mask, label])
            n_corr = n0 - pop_other * n_unas / n_as
            pred_corr.append(n_corr)
    mean, delta_plus, delta_minus = aux.get_mean_dp_dm(pred)
    print('Number of %s: $%.1f^{+%.1f}_{-%.1f}$' % (pop, mean, delta_plus, delta_minus))
    if nclasses == 2:
        mean_corr, delta_plus, delta_minus = aux.get_mean_dp_dm(pred_corr)
        print('Number of %s (corr): $%.1f^{+%.1f}_{-%.1f}$' % (pop, mean_corr, delta_plus, delta_minus))
        noc += mean - mean_corr
if nclasses == 2:
    n_other = np.sum(other_mask) * n_unas / n_as
    print('n other model: %.1f' % n_other)
    print(noc)

Number of AGN: $664.0^{+74.9}_{-65.2}$
Number of PSR: $158.3^{+22.3}_{-23.6}$
Number of OTHER: $187.6^{+46.4}_{-51.3}$


In [16]:
#cat_df.columns

In [17]:
mv_mask = (cat_df['Missing_Values_Flag'] == 1)
np.sum(cat_df[mv_mask]['Category_%s' % cat] == 'UNAS')

2